# FAISS basics

FAISS (Facebook AI Similarity Search) is a library for efficient similarity search and clustering of dense vectors. It is widely used in applications involving nearest-neighbor search and large-scale machine learning. Common use cases include image search (finding similar images based on features), text search using embeddings (e.g., semantic similarity) and clustering high-dimensional data.

#### Core concept: Vector similarity search
FAISS works with dense vectors. For example, a vector can represent:
- An image (via image embeddings).
- A document (via text embeddings like BERT).

The library finds the nearest vectors (most similar vectors) to a given query vector.

#### Key components:
1. **Index:** A data structure used to store vectors for search.
2. **Search:** A method to retrieve the nearest vectors based on a similarity metric (e.g., Euclidean or cosine similarity).

In [1]:
import faiss
import numpy as np

#### Example dataset
Let’s create a small dataset of random vectors to simulate how FAISS works.

In [2]:
# Generate a dataset of random vectors
# Assume we have 1000 vectors, each of dimension 128 (common in embeddings)
dimension = 128  # Length of each vector
num_vectors = 1000  # Number of vectors in the dataset

# Randomly generate vectors
data = np.random.random((num_vectors, dimension)).astype('float32')  # FAISS requires float32

print("Shape of dataset:", data.shape)

Shape of dataset: (1000, 128)


## Indexing strategies

### IndexFlatL2
FAISS provides several types of indices. The simplest one is the **IndexFlatL2**, which performs brute-force search using the L2 (Euclidean) distance. It stores all the data (vectors) in memory so that when we want to search for something, it directly compares our query with every vector in the dataset. Since it checks each vector individually, the search is guaranteed to be accurate, but it can be slower for larger datasets. IndexFlatL2 is ideal for small datasets, such as those with thousands of vectors, where precision is more important than speed.

The "flat" in the name means that all the vectors are stored in a simple, non-hierarchical way without any advanced indexing techniques such as additional structure like clustering or partitioning. This makes the structure "flat," where every vector is directly accessible and compared individually during the search.

#### Step 1: Initialize the index
Initializing an index in FAISS means creating a data structure that will store vectors for the purpose of similarity search.

In [3]:
# Step 1: Initialize an index
index = faiss.IndexFlatL2(dimension)  # L2 indicates Euclidean distance
print("Is the index trained?", index.is_trained)  # Flat indices are always trained

Is the index trained? True


In this case, we initialize an index that uses the L2 (Euclidean) distance to measure similarity. The `dimension` specifies the length of each vector, so the index knows what size each vector should be.

In FAISS, training refers to the process of preparing an index to be used for efficient search. Some types of indices require training on a sample of data to learn how to organize the vectors for fast search. However, for simple indices like IndexFlatL2, which simply store the vectors in memory and perform a brute-force search, training is not required.

#### Step 2: Add vectors to the index
When we add vectors to the index in FAISS, we are essentially storing the vectors (e.g., image embeddings or text embeddings) in the index so that they can later be used for similarity searches.

In [4]:
# Step 2: Add vectors to the index
index.add(data)  # Add the dataset to the index
print("Number of vectors in the index:", index.ntotal)

Number of vectors in the index: 1000


By adding the vectors to the index, we enable the index to perform efficient searches when we query it, comparing the query vector to those stored in the index to find the closest matches based on the similarity metric (e.g., L2 distance).

#### Step 3: Perform a search
Here, we will perform a search to find the nearest neighbors of a given query vector in the index.

In [5]:
# Step 3: Perform a search
# Create a random query vector (of the same dimension as the dataset)
query_vector = np.random.random((1, dimension)).astype('float32')

# Search for the top 5 nearest neighbors
k = 5  # Number of nearest neighbors
distances, indices = index.search(query_vector, k)

# Display results
print("Distances to nearest neighbors:", distances)
print("Indices of nearest neighbors:", indices)

Distances to nearest neighbors: [[14.414135 15.166716 16.051807 16.167038 16.283192]]
Indices of nearest neighbors: [[558 782 318 735 283]]


The query vector must have the same dimension as the vectors in the dataset (in this case, 128), because the index stores vectors of that size.
- `distances`: This variable will contain the distances (e.g., Euclidean distance) between the query vector and the 5 nearest vectors found in the index.
- `indices`: This variable contains the indices (or positions) of the nearest vectors in the index. These indices tell us where in the dataset the closest vectors are located.

### IndexIVFFlat (Inverted file index + flat search)
IndexIVFFlat is an advanced indexing technique that improves search speed by dividing the dataset into smaller groups (clusters) using a clustering algorithm. Instead of comparing a query to every vector in the dataset, the query is compared only to vectors in the most relevant cluster(s). This reduces the number of comparisons, making it much faster for large datasets. However, it is an approximate method, so results may not be as accurate as brute-force (IndexFlatL2).

#### Step 1: Create an IVF index
Here, we are setting up an indexing structure that partitions the dataset into smaller groups to make searching faster.

In [6]:
# Step 1: Create an IVF index
nlist = 100  # Number of clusters (partitions)
quantizer = faiss.IndexFlatL2(dimension)  # Quantizer for clustering
ivf_index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)

The choice of the number of clusters depends on how large the dataset is and the balance between speed and accuracy. A larger number of clusters can improve accuracy but might increase memory usage and search time. A smaller number of clusters makes the index more efficient but could lead to less accurate results.

The quantizer is a technique used to perform clustering. In this case, `faiss.IndexFlatL2(dimension)` is used as the quantizer, which means the vectors will be grouped based on L2 (Euclidean) distance. The quantizer helps FAISS group similar vectors together by measuring how close the vectors are to each other.

#### Step 2: Train the index
Training the index is an essential step when using an IVF index, as it learns how to organize data into clusters. To do this, it needs a set of representative data — a sample of the dataset that reflects the general distribution of the vectors, so that the index can effectively divide the vectors into meaningful clusters.



In [7]:
# Step 2: Train the index
ivf_index.train(data)  # Training requires representative data
print("Is the IVF index trained?", ivf_index.is_trained)

Is the IVF index trained? True


During training, the IVF index uses the training data (in this case, `data`) to find the best way to partition the vector space into clusters. When we call `ivf_index.train(data)`, FAISS uses the k-means algorithm to analyze the data and figure out how to split it into clusters. After training, the IVF index will store these centroids (representing each cluster), and when we perform a search, the query is first compared to the centroids of the clusters.

#### Step 3: Add vectors to the index
In this step, the vectors are added to the IVF index that we have created and trained. The IVF index does not store vectors in a flat structure like IndexFlatL2. Instead, it organizes them into clusters.

In [8]:
# Step 3: Add vectors to the index
ivf_index.add(data)
print("Number of vectors in the IVF index:", ivf_index.ntotal)

Number of vectors in the IVF index: 1000


The vectors that are added are placed into the appropriate clusters that were determined during the training step (they are assigned to the closest cluster centroid). After this step, the IVF index will have access to the vectors, and it can perform similarity searches on them.

### Step 4: Perform a search
In this step, we use the trained IVF index to perform a similarity search for a given query vector.

In [9]:
# Step 4: Perform a search
distances, indices = ivf_index.search(query_vector, k)  # Search for top-k neighbors
print("IVF Distances to nearest neighbors:", distances)
print("IVF Indices of nearest neighbors:", indices)

IVF Distances to nearest neighbors: [[16.167038 16.741135 16.834538 17.297047 17.438253]]
IVF Indices of nearest neighbors: [[735 697 421 995 593]]


Here, we search for the top-k nearest neighbors to the query_vector in the IVF index. The query vector is first compared to the centroids of the clusters in the IVF index. Then, the query is only compared to vectors in the most relevant clusters and the top-k most similar vectors (neighbors) are found based on a similarity metric (e.g., Euclidean distance).
- `distances` shows the closeness of the query vector to the neighbors.
- `indices` shows the actual positions of the neighbors in the original dataset.

### IndexIVFPQ (Inverted file index + product quantization)
IndexIVFPQ combines the clustering of IndexIVFFlat with product quantization (PQ), a compression technique that reduces the memory footprint of each vector. PQ breaks each vector into smaller chunks and compresses them into smaller representations. This method is highly memory-efficient and faster, though it introduces more approximation.

In IVF-PQ, vectors are divided into clusters (using IVF), and each vector within a cluster is compressed (using PQ) to reduce memory usage. PQ works by breaking each high-dimensional vector into smaller parts and representing them with fewer bits, thus reducing the storage needed for each vector.

#### Step 1: Create an IVF-PQ index


In [10]:
# Step 1: Create an IVF-PQ index
m = 8  # Number of chunks for PQ
ivfpq_index = faiss.IndexIVFPQ(quantizer, dimension, nlist, m, 8)  # (m, 8) -> PQ compression level

`m` is the number of chunks that the vector will be divided into during the PQ process. For example, if we have a vector of dimension 128 and set `m=8`, each vector will be divided into 8 chunks, with each chunk representing a part of the original vector. More chunks (`m`) means more compression, but each chunk will carry less information (and the approximation will be higher).

The second `8` in the constructor is the quantization level (the number of bits used to represent each chunk) and it determines how much memory is saved. Typically, a smaller number of bits for each chunk means higher compression but more approximation, as the vector will be represented less precisely. For example, `8` means each chunk will be represented with 8 bits, while a larger number might mean less compression (but more accuracy).

#### Step 2: Train the index
In this step, we are training the IVF-PQ index on the provided dataset. The purpose of this training is to prepare the index for efficient similarity searches, involving both clustering (IVF) and compression (PQ).


In [11]:
# Step 2: Train the index
ivfpq_index.train(data)  # Training for clustering and PQ compression
print("Is the IVFPQ index trained?", ivfpq_index.is_trained)

Is the IVFPQ index trained? True


During the training, the IVF part of the index learns to divide the dataset into clusters. In addition, the training process also learns how to compress the vectors using PQ. The PQ process divides each vector into chunks (as defined by `m`), and it calculates how to compress each chunk efficiently. Clustering (IVF) and compression (PQ) require training because they need to understand the structure of the data (how the vectors are distributed and how they can be compressed).

#### Step 3: Add vectors to the index
After training the index, the next step is to add the actual data vectors into the index.

In [12]:
# Step 3: Add vectors to the index
ivfpq_index.add(data)
print("Number of vectors in the IVFPQ index:", ivfpq_index.ntotal)

Number of vectors in the IVFPQ index: 1000


Each vector will be placed into its appropriate cluster (determined during training) based on which cluster centroid is closest. PQ is used to compress each vector as it’s added to the index, reducing memory usage, as was determined during the training phase.

#### Step 4: Perform a search
In this step, we are performing a search on the IVF-PQ index to find the nearest neighbors to a given query vector.

In [13]:
# Step 4: Perform a search
distances, indices = ivfpq_index.search(query_vector, k)  # Search for top-k neighbors
print("IVFPQ Distances to nearest neighbors:", distances)
print("IVFPQ Indices of nearest neighbors:", indices)

IVFPQ Distances to nearest neighbors: [[12.494565  13.5019045 13.937946  14.385806  14.43169  ]]
IVFPQ Indices of nearest neighbors: [[735 995 593 203 280]]


First, the search checks which cluster(s) the query vector likely belongs to (based on the trained centroids from the IVF step). Since the index is using PQ, each vector is stored in a compressed form. During the search, these compressed representations are used.

### IndexScalarQuantizer (Scalar quantization)
IndexScalarQuantizer compresses vectors using scalar quantization. Each vector dimension is quantized (discretized) into a small number of values (quantization levels), reducing memory usage. Unlike PQ, which compresses chunks of dimensions, scalar quantization operates dimension by dimension. This is simpler and less memory-efficient than PQ, but it can still speed up searches while saving memory.

Instead of using the exact values for each dimension, scalar quantization maps each dimension to a fixed number of levels (like rounding to a specific number of values).For example, if we have a vector with values like `[0.2, 0.5, 0.8]`, scalar quantization might map each value to a small set of possible values, such as `[0, 0.5, 1]`. This process reduces the precision of the vector, saving memory.

#### Step 1: Create a ScalarQuantizer index

In [14]:
# Step 1: Create a ScalarQuantizer index
scalar_quantizer = faiss.ScalarQuantizer(dimension, faiss.ScalarQuantizer.QT_8bit)  # 8-bit quantization
sq_index = faiss.IndexIVFScalarQuantizer(quantizer, dimension, nlist, scalar_quantizer.qtype, faiss.METRIC_L2)

The `faiss.ScalarQuantizer.QT_8bit` specifies that each dimension will be quantized to 8 bits, meaning each dimension can take one of 256 possible values (2^8).

#### Step 2: Train the index

In [15]:
# Step 2: Train the index
sq_index.train(data)
print("Is the ScalarQuantizer index trained?", sq_index.is_trained)

Is the ScalarQuantizer index trained? True


#### Step 3: Add vectors to the index

In [16]:
# Step 3: Add vectors to the index
sq_index.add(data)
print("Number of vectors in the ScalarQuantizer index:", sq_index.ntotal)

Number of vectors in the ScalarQuantizer index: 1000


#### Step 4: Perform a search

In [17]:
# Step 4: Perform a search
distances, indices = sq_index.search(query_vector, k)  # Search for top-k neighbors
print("ScalarQuantizer Distances to nearest neighbors:", distances)
print("ScalarQuantizer Indices of nearest neighbors:", indices)

ScalarQuantizer Distances to nearest neighbors: [[16.180897 16.732254 16.82972  17.307941 17.425316]]
ScalarQuantizer Indices of nearest neighbors: [[735 697 421 995 593]]
